## Basic Momentum Strategy

The fundaments of this strategy is pretty simple: __buy and hold stocks within the top-decile of 12-month lagged returns for 1M and vice versa for the bottom decile__

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import yfinance as yf

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

import sys
sys.path.append('/Users/educontreras/PycharmProjects/Quantitative-Finance/AlgoTrading_E_Chang')
import alpha_vantage
from Notebooks.utils import utils
from Notebooks.utils.config import alpha_vantage_api_key as av_key

In [71]:
def compute_spx_basic_momentum_strategy(prices_df = None,start_date ="2005-01-01",end_date =datetime.today().strftime("%Y-%m-%d") ,lookback = 252, holding_days = 22, top_quintile = 0.1):
    
    if prices_df is None:
        prices_df = utils.get_spx_constituents_price(start_date)
    topN = int(round(prices_df.shape[1]*top_quintile))
    ret = prices_df.pct_change(lookback)
    longs = np.full(prices_df.shape, False)
    shorts = np.full(prices_df.shape, False)
    positions = np.zeros(prices_df.shape)
    for t in range(lookback,prices_df.shape[0]):
        hasData = np.where(np.isfinite(ret.iloc[t, : ]))
        hasData = hasData[0]
        if len(hasData)>0:
            #Interesting method to sort the elements of a given pandas.Series
            idxSort = np.argsort(ret.iloc[t,hasData])
            best_perf =  hasData[idxSort.values[np.arange(-np.min((topN, len(idxSort))),0)]]
            worst_perf = hasData[idxSort.values[np.arange(0,topN)]]
            longs[t,best_perf] = 1
            shorts[t, worst_perf] = 1
    longs = pd.DataFrame(longs)
    shorts = pd.DataFrame(shorts)
    #Knowing the starting for longs and shorts we only need to fill positions matrix for every day we hold each asset. 
    for h in range(holding_days - 1):
        long_lag = longs.shift(h).fillna(False)
        short_lag = shorts.shift(h).fillna(False)
        positions[long_lag] = positions[long_lag] + 1
        #positions[short_lag] = positions[short_lag] - 1

    positions = pd.DataFrame(
        index = prices_df.index,
        columns = prices_df.columns,
        data = positions)
    ret=pd.DataFrame((positions.shift(2).values)*(prices_df.pct_change().values)).fillna(0).sum(axis = 1)/(2*topN)/holding_days# daily P&L of the strategy
    cumret=(np.cumprod(1+ret)-1)
    cumret.index = prices_df.index
    return cumret, np.prod(1+ret)**(252/len(ret))-1, np.sqrt(252)*np.mean(ret)/np.std(ret)


In [ ]:
pnl,r,s = compute_spx_basic_momentum_strategy(start_date = "2010-12-01")
pnl.iplot()

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-12-01 -> 2024-04-04)')


In [ ]:
s